# understanding data cleaning part 1

This file attempts to walkthrough some of the data cleaning performed by last year's project to get a better understanding of the data used for the analysis. The point of the cleaning is to get a better understanding of the enrollment size for each district and county.

In [5]:
## loading in libraries for analysis
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [6]:
## loading in the data
k12 = pd.read_excel("OH_K12.xlsx", sheet_name = "OH_K12")

In [7]:
k12.shape

(2786, 35)

In [8]:
k12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2786 entries, 0 to 2785
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   SrcName                          2786 non-null   object        
 1   ncessch                          2786 non-null   int64         
 2   schnam                           2786 non-null   object        
 3   leaid                            2786 non-null   int64         
 4   gslo                             2786 non-null   object        
 5   gshi                             2786 non-null   object        
 6   STUSAB                           2786 non-null   object        
 7   openEnroll                       2786 non-null   int64         
 8   level                            2786 non-null   object        
 9   MultiBdy                         2786 non-null   int64         
 10  CITY                             2736 non-null   object     

In [9]:
k12.isna().sum()

SrcName                              0
ncessch                              0
schnam                               0
leaid                                0
gslo                                 0
gshi                                 0
STUSAB                               0
openEnroll                           0
level                                0
MultiBdy                             0
CITY                                50
COUNTY                              50
LEVEL_                              50
ENROLLMENT                          50
SchoolYear                           4
DistrictName                         4
Control                              4
PhysicalCity                         4
PhysicalState                        4
Enrollment_1                         4
DATE                               168
TeachingMethod                       4
SportsParticipation                  4
OnlineInstructionIncrease            4
NetworkInvestment                    4
HardwareInvestment       

## removing unnecessary columns

After exploring the data, we find that some of the columns contain minimal or redundant information and are therefore removed.

SrcName = redundant information, many missing values

ncessch = redundant information, only two values

CITY = redundant information with another column (PhysicalCity) and has more missing values, based on documentation CITY (EXACT_CITY in the file) is contained with PhysicalCity in some instances as well

STUSAB, PhysicalState = not informative (state is Ohio)

SchoolYear, Control = not informative columns, only includes year (2020-2021) and school type (public)

In [10]:
print(k12[k12.SrcName != " "].shape) # many values are blank

(842, 35)


In [11]:
# only two values, redundant with other information
k12.ncessch.value_counts()

390000000000    2452
391000000000     334
Name: ncessch, dtype: int64

In [12]:
# remove becuase only one value
print(k12.STUSAB.unique()) 
print(k12.PhysicalState.unique()) 

['OH']
['OH' nan]


In [13]:
k12.SchoolYear.unique() # remove because only one value

array(['2020-2021', nan], dtype=object)

In [14]:
k12.Control.unique() # remove because only one value or NaN

array(['Public', nan], dtype=object)

In [15]:
# same information captured by LEVEL_ and level, LEVEL_ dropped because redundant
# see documentation for mapping of codes
print(k12.LEVEL_.unique())
print(k12.level.unique())

['HIGH' 'ELEMENTARY' 'NOT REPORTED' 'MIDDLE' nan 'OTHER' 'SECONDARY'
 'NOT APPLICABLE' 'PREKINDERGARTEN']
[3 1 2 4 'N']


In [16]:
# redundant information between leaid and District_nces (school identifiers), District_nces will be dropped since incomplete
print(len(set(k12.leaid)))
print(len(set(k12.District_nces)))
print(set(k12.leaid).difference(k12.District_nces))
print(set(k12.District_nces).difference(k12.leaid))

572
573
{3906496, 3904721, 3904679}
{nan, nan, nan, nan}


In [17]:
## removing the unhelpful/redundant variables
redundant = ["SrcName", "ncessch", "STUSAB", "CITY", "LEVEL_", "PhysicalState", "SchoolYear", "District_nces", "Control"]
k12_1 = k12.drop(columns = redundant, axis = 1)
k12_1.shape

(2786, 26)

## exploring the explicit nas

First, look at columns with explicit Na values of the remaining columns. Some columns have similar amounts of NAs, and it turns out that these correspond to the same indexes (row numbers). From the explicit Na values, 217 total are explicitly missing, which is abour 8% of the observations in the data set.

In [18]:
k12_1.isnull().sum()

schnam                               0
leaid                                0
gslo                                 0
gshi                                 0
openEnroll                           0
level                                0
MultiBdy                             0
COUNTY                              50
ENROLLMENT                          50
DistrictName                         4
PhysicalCity                         4
Enrollment_1                         4
DATE                               168
TeachingMethod                       4
SportsParticipation                  4
OnlineInstructionIncrease            4
NetworkInvestment                    4
HardwareInvestment                   4
StaffMaskPolicy                      4
StudentMaskPolicy                    4
StudentIllnessReturnPolicy           4
StudentIsolationArea                 4
SchoolTemporaryShutdown              4
ParentOptOutClassroomTeachin_55      4
LastVerifiedDate                   168
OpenDateGrouped          

In [19]:
## checking if the same indices all have NaNs for specified columns - 1882, 1883, 1884, 2785 are missing
for col in k12_1[k12_1.isnull().sum().index[k12_1.isnull().sum() == 4].tolist()]:
    tmp = k12_1[col]
    print("col name ", col)
    print("indexes with NaNs", tmp[tmp.isnull()].index)
    print("\n")

col name  DistrictName
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  PhysicalCity
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  Enrollment_1
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  TeachingMethod
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  SportsParticipation
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  OnlineInstructionIncrease
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  NetworkInvestment
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  HardwareInvestment
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  StaffMaskPolicy
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name  StudentMaskPolicy
indexes with NaNs Int64Index([1882, 1883, 1884, 2785], dtype='int64')


col name 

In [20]:
k12_1.iloc[1882,:]

schnam                             Kelleys Island High School
leaid                                                 3904679
gslo                                                        N
gshi                                                        N
openEnroll                                                  1
level                                                       N
MultiBdy                                                    0
COUNTY                                                   ERIE
ENROLLMENT                                                1.0
DistrictName                                              NaN
PhysicalCity                                              NaN
Enrollment_1                                              NaN
DATE                                                      NaT
TeachingMethod                                            NaN
SportsParticipation                                       NaN
OnlineInstructionIncrease                                 NaN
NetworkI

In [21]:
# enrollment/county have same missing values
print(len(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index).intersection(set(k12_1.COUNTY[k12_1.COUNTY.isnull()].index))))
print(len(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index))) # complete overlap of missing values

50
50


In [22]:
## time NaTs - all have the same times (168)
print(len(set(k12_1.DATE[k12_1.DATE.isnull()].index)))
print(len(set(k12_1.OpenDateGrouped[k12_1.OpenDateGrouped.isnull()].index)))
print(len(set(k12_1.LastVerifiedDate[k12_1.LastVerifiedDate.isnull()].index)))
print(len(set(k12_1.DATE[k12_1.DATE.isnull()].index).intersection(set(k12_1.LastVerifiedDate[k12_1.LastVerifiedDate.isnull()].index))))
print(len(set(k12_1.DATE[k12_1.DATE.isnull()].index).intersection(set(k12_1.OpenDateGrouped[k12_1.OpenDateGrouped.isnull()].index))))
print(len(set(k12_1.OpenDateGrouped[k12_1.OpenDateGrouped.isnull()].index).intersection(set(k12_1.OpenDateGrouped[k12_1.OpenDateGrouped.isnull()].index))))

168
168
168
168
168
168


In [23]:
print(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index).intersection(set(k12_1.DATE[k12_1.DATE.isnull()].index)))

{133}


In [24]:
## checking overlap of the missing values - 5 values overlap between all Nas 133, 1882, 1883, 1884, 2785 repeat 
print(len(set(k12_1.DATE[k12_1.DATE.isnull()].index)) + len(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index)) + len(set(k12_1.DistrictName[k12_1.DistrictName.isnull()].index)))
print(len(set(k12_1.DATE[k12_1.DATE.isnull()].index).union(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index)).union(set(k12_1.DistrictName[k12_1.DistrictName.isnull()].index))))

222
217


In [25]:
round(len(set(k12_1.DATE[k12_1.DATE.isnull()].index).union(set(k12_1.ENROLLMENT[k12_1.ENROLLMENT.isnull()].index)).union(set(k12_1.DistrictName[k12_1.DistrictName.isnull()].index)))/k12_1.shape[0], 4)

0.0779

## exploring the other columns with nas that are not explicit

some columns have unusual values that are noted, and some are set to NaN. The NaN investigation also shows that there are instances of the same row having multiple NaNs across the columns.

In [26]:
k12_1.schnam.value_counts() # Unassigned = na? okay for now

Unassigned                         28
Washington Elementary School        8
Lincoln Elementary School           7
Garfield Elementary School          6
South Elementary School             5
                                   ..
Theodore Roosevelt High School      1
Hilliard Heritage Middle School     1
Highview 6th Grade Center           1
West Geauga High School             1
Alpine Elementary School            1
Name: schnam, Length: 2571, dtype: int64

In [27]:
k12_1.level.value_counts() # N = NaN?
# not applicable per documentation

1    1468
2     626
3     621
4      61
N      10
Name: level, dtype: int64

In [28]:
k12_1.level.index[k12_1.level == "N"]

Int64Index([49, 224, 225, 583, 878, 958, 965, 1060, 1457, 1882], dtype='int64')

In [29]:
k12_1.gslo.value_counts() # N = NaN? - missing data

KG    738
PK    570
9     475
6     334
7     222
5     148
3      96
4      65
2      52
8      42
1      32
N      10
10      1
11      1
Name: gslo, dtype: int64

In [30]:
k12_1.gslo.index[k12_1.gslo == "N"]

Int64Index([49, 224, 225, 583, 878, 958, 965, 1060, 1457, 1882], dtype='int64')

In [31]:
k12_1.gshi.value_counts() # N = NaN? - missing data

12    662
5     612
8     590
6     384
4     252
2      89
3      87
1      30
9      24
7      19
KG     14
10     10
N      10
11      3
Name: gshi, dtype: int64

In [43]:
## all have the same indexes with missing values - Na values
print(len(set(k12_1.level.index[k12_1.level == "N"]).intersection(set(k12_1.gslo.index[k12_1.gslo == "N"]))))
print(len(set(k12_1.gshi.index[k12_1.gshi == "N"]).intersection(set(k12_1.gslo.index[k12_1.gslo == "N"]))))
print(len(set(k12_1.gshi.index[k12_1.gshi == "N"]).intersection(set(k12_1.level.index[k12_1.level == "N"]))))
print(len(set(k12_1.gshi.index[k12_1.gshi == "N"]).intersection(set(k12_1.level.index[k12_1.level == "N"])).intersection(set(k12_1.gslo.index[k12_1.gslo == "N"]))))
k12_1.gslo.index[k12_1.gslo == "N"].tolist()

0
0
0
0


[]

In [44]:
# setting these values to Na
k12_1["level"] = k12_1.loc[:, "level"].replace({"N":np.nan})
k12_1["gshi"] = k12_1.loc[:, "gshi"].replace({"N":np.nan})
k12_1["gslo"] = k12_1.loc[:, "gslo"].replace({"N":np.nan})

In [45]:
k12_1.isnull().sum()

schnam                               0
leaid                                0
gslo                                10
gshi                                10
openEnroll                         112
level                               10
MultiBdy                             0
COUNTY                              50
ENROLLMENT                          75
DistrictName                         4
PhysicalCity                         4
Enrollment_1                         4
DATE                               168
TeachingMethod                       4
SportsParticipation                  4
OnlineInstructionIncrease            4
NetworkInvestment                    4
HardwareInvestment                   4
StaffMaskPolicy                      4
StudentMaskPolicy                    4
StudentIllnessReturnPolicy           4
StudentIsolationArea                 4
SchoolTemporaryShutdown              4
ParentOptOutClassroomTeachin_55      4
LastVerifiedDate                   168
OpenDateGrouped          

In [46]:
k12_1.openEnroll.value_counts() # 2 = NaN? - missing data (not listed in documentation)

0.0    2641
1.0      33
Name: openEnroll, dtype: int64

In [47]:
k12_1["openEnroll"] = k12_1.loc[:, "openEnroll"].replace({2:np.nan})

In [48]:
print(k12_1.ENROLLMENT[k12_1.ENROLLMENT==k12_1.ENROLLMENT.min()])
k12_1.ENROLLMENT[k12_1.ENROLLMENT==k12_1.ENROLLMENT.min()].shape # 25 schools have negative enrollment - NaN?

1882    1.0
Name: ENROLLMENT, dtype: float64


(1,)

In [49]:
k12_1["ENROLLMENT"] = k12_1.loc[:, "ENROLLMENT"].replace({k12_1.ENROLLMENT.min():np.nan})

## imputing counties

idea: there are counties with missing data but no cities with missing data, so we try to impute the missing values by using the known city known in each county for the missing data points. We first separate data into county-city pairs, then isolate the missing counties. Then, we combine these data sets to try to find the appropriate county with missing data based on the county-city pairs. This is not perfect but a rough approximation of the counties.

In [50]:
 k12_1.groupby(["COUNTY", "PhysicalCity"]).size().reset_index()

,COUNTY,PhysicalCity,0
0,ADAMS,Manchester,2
1,ADAMS,West Union,6
2,ALLEN,Bluffton,3
3,ALLEN,Delphos,2
4,ALLEN,Elida,3
...,...,...,...
500,WOOD,Perrysburg,6
501,WOOD,Rossford,4
502,WYANDOT,Carey,2
503,WYANDOT,Sycamore,2


In [51]:
pair = k12_1.groupby(["COUNTY", "PhysicalCity"]).size().reset_index()[["COUNTY", "PhysicalCity"]]
pair

,COUNTY,PhysicalCity
0,ADAMS,Manchester
1,ADAMS,West Union
2,ALLEN,Bluffton
3,ALLEN,Delphos
4,ALLEN,Elida
...,...,...
500,WOOD,Perrysburg
501,WOOD,Rossford
502,WYANDOT,Carey
503,WYANDOT,Sycamore


In [52]:
miss_county = k12_1[k12_1.COUNTY.isnull()][["schnam", "COUNTY", "PhysicalCity", "MultiBdy"]]
miss_county.reset_index(inplace = True)

In [53]:
miss_county

,index,schnam,COUNTY,PhysicalCity,MultiBdy
0,25,Lawndale Elementary School,NaN,Akron,0
1,39,Smith Elementary School,NaN,Akron,0
2,49,Akron Opportunity Center,NaN,Akron,0
3,106,Unassigned,NaN,Berea,0
4,133,Unassigned,NaN,Brunswick,0
5,136,Lincoln Elementary School,NaN,Bryan,0
6,224,Western Hills Engineering High School,NaN,Cincinnati,0
7,225,Withrow International High School,NaN,Cincinnati,0
8,245,Park Elementary School,NaN,Dennison,0
9,404,Frank B Willis Intermediate Middle School,NaN,Delaware,0


In [54]:
t3 = miss_county.merge(pair, how = "left", on = "PhysicalCity")
t3.shape # number of missing counties is not the same after the merge ==> implies duplicates present

(58, 6)

In [55]:
t3.drop_duplicates("index") # removes duplicates for imputing

,index,schnam,COUNTY_x,PhysicalCity,MultiBdy,COUNTY_y
0,25,Lawndale Elementary School,NaN,Akron,0,SUMMIT
1,39,Smith Elementary School,NaN,Akron,0,SUMMIT
2,49,Akron Opportunity Center,NaN,Akron,0,SUMMIT
3,106,Unassigned,NaN,Berea,0,CUYAHOGA
4,133,Unassigned,NaN,Brunswick,0,MEDINA
5,136,Lincoln Elementary School,NaN,Bryan,0,WILLIAMS
6,224,Western Hills Engineering High School,NaN,Cincinnati,0,CLERMONT
8,225,Withrow International High School,NaN,Cincinnati,0,CLERMONT
10,245,Park Elementary School,NaN,Dennison,0,TUSCARAWAS
11,404,Frank B Willis Intermediate Middle School,NaN,Delaware,0,DELAWARE


In [56]:
t3.groupby("PhysicalCity")["COUNTY_y"].nunique()[t3.groupby("PhysicalCity")["COUNTY_y"].nunique() > 1]
# duplicates present in Cincinnati and Dublin

PhysicalCity
Cincinnati    2
Dublin        3
Name: COUNTY_y, dtype: int64

dublin: https://en.wikipedia.org/wiki/Dublin,_Ohio

multiple cities named dublin in the state of Ohio (Union, Franklin, Delaware)

cincinnati: https://en.wikipedia.org/wiki/Cincinnati_metropolitan_area

there are multiple counties within Cincinnati (includes 6 total per wikipedia)

In [57]:
k12_1.isnull().sum()

schnam                               0
leaid                                0
gslo                                10
gshi                                10
openEnroll                         112
level                               10
MultiBdy                             0
COUNTY                              50
ENROLLMENT                          76
DistrictName                         4
PhysicalCity                         4
Enrollment_1                         4
DATE                               168
TeachingMethod                       4
SportsParticipation                  4
OnlineInstructionIncrease            4
NetworkInvestment                    4
HardwareInvestment                   4
StaffMaskPolicy                      4
StudentMaskPolicy                    4
StudentIllnessReturnPolicy           4
StudentIsolationArea                 4
SchoolTemporaryShutdown              4
ParentOptOutClassroomTeachin_55      4
LastVerifiedDate                   168
OpenDateGrouped          

In [58]:
# this imputes (replaces) the NAs by county values in the merged dataset
k12_1.iloc[t3["index"].drop_duplicates(), 7] = t3.drop_duplicates("index").COUNTY_y

In [61]:
k12_1.TeachingMethod[k12_1.TeachingMethod.isnull()]

1882    NaN
1883    NaN
1884    NaN
2785    NaN
Name: TeachingMethod, dtype: object

In [62]:
k12_1.shape

(2786, 26)

In [63]:
k12_1.iloc[1882,:]

schnam                             Kelleys Island High School
leaid                                                 3904679
gslo                                                      NaN
gshi                                                      NaN
openEnroll                                                1.0
level                                                     NaN
MultiBdy                                                    0
COUNTY                                                   ERIE
ENROLLMENT                                                NaN
DistrictName                                              NaN
PhysicalCity                                              NaN
Enrollment_1                                              NaN
DATE                                                      NaT
TeachingMethod                                            NaN
SportsParticipation                                       NaN
OnlineInstructionIncrease                                 NaN
NetworkI

In [64]:
k12_1.iloc[1883,:]

schnam                             Newbury Elementary School
leaid                                                3904721
gslo                                                      KG
gshi                                                       5
openEnroll                                               0.0
level                                                    1.0
MultiBdy                                                   0
COUNTY                                                GEAUGA
ENROLLMENT                                             136.0
DistrictName                                             NaN
PhysicalCity                                             NaN
Enrollment_1                                             NaN
DATE                                                     NaT
TeachingMethod                                           NaN
SportsParticipation                                      NaN
OnlineInstructionIncrease                                NaN
NetworkInvestment       

In [65]:
k12_1.iloc[1884,:]

schnam                             Newbury High School
leaid                                          3904721
gslo                                                 6
gshi                                                12
openEnroll                                         0.0
level                                              4.0
MultiBdy                                             0
COUNTY                                          GEAUGA
ENROLLMENT                                       233.0
DistrictName                                       NaN
PhysicalCity                                       NaN
Enrollment_1                                       NaN
DATE                                               NaT
TeachingMethod                                     NaN
SportsParticipation                                NaN
OnlineInstructionIncrease                          NaN
NetworkInvestment                                  NaN
HardwareInvestment                                 NaN
StaffMaskP

In [66]:
k12_1.iloc[2785,:]

schnam                             College Corner Union Elementary School
leaid                                                             3906496
gslo                                                                   KG
gshi                                                                   12
openEnroll                                                            0.0
level                                                                 4.0
MultiBdy                                                                0
COUNTY                                                             PREBLE
ENROLLMENT                                                          105.0
DistrictName                                                          NaN
PhysicalCity                                                          NaN
Enrollment_1                                                          NaN
DATE                                                                  NaT
TeachingMethod                        

## calculating enrollment

Now that we've imputted the data and cleaned it, we can calculate enrollment by district and then by county. Since there are two measures for enrollment, we compare the two and take the larger of them. Note that ENROLLMENT is by school, while Enrollment_1 is by school district, so the larger of the two is used.

After finding enrollment by school district, we aggregate this up by county and then recombine all the data.

In [45]:
# missing value handling for enrollment
# Enrollment_1 vs ENROLLMENT comparison - take the larger of the two
k12_1.dropna(subset = ["TeachingMethod"], inplace = True)

In [47]:
enroll = k12_1.groupby(["COUNTY", "leaid", "Enrollment_1"])["ENROLLMENT"].sum().reset_index()
enroll.head()

,COUNTY,leaid,Enrollment_1,ENROLLMENT
0,ADAMS,3900537,932.0,879.0
1,ADAMS,3906190,4044.0,3612.0
2,ALLEN,3904388,885.0,442.0
3,ALLEN,3904422,4223.0,3625.0
4,ALLEN,3904521,1220.0,1153.0


In [48]:
enroll.shape

(587, 4)

In [49]:
enroll['district_enroll'] = enroll[['Enrollment_1', 'ENROLLMENT']].max(axis = 1) #.values.max(1) as alternative (slighly faster)

In [50]:
enroll.head()

,COUNTY,leaid,Enrollment_1,ENROLLMENT,district_enroll
0,ADAMS,3900537,932.0,879.0,932.0
1,ADAMS,3906190,4044.0,3612.0,4044.0
2,ALLEN,3904388,885.0,442.0,885.0
3,ALLEN,3904422,4223.0,3625.0,4223.0
4,ALLEN,3904521,1220.0,1153.0,1220.0


In [51]:
enroll2 = enroll.groupby("COUNTY")["district_enroll"].sum().reset_index()
enroll2

,COUNTY,district_enroll
0,ADAMS,4976.0
1,ALLEN,16319.0
2,ASHLAND,7452.0
3,ASHTABULA,12031.0
4,ATHENS,7412.0
...,...,...
81,WASHINGTON,5312.0
82,WAYNE,14429.0
83,WILLIAMS,6076.0
84,WOOD,20781.0


In [52]:
enroll2.rename(columns = {"district_enroll":"county_enroll"}, inplace = True)

In [53]:
enroll.head()

,COUNTY,leaid,Enrollment_1,ENROLLMENT,district_enroll
0,ADAMS,3900537,932.0,879.0,932.0
1,ADAMS,3906190,4044.0,3612.0,4044.0
2,ALLEN,3904388,885.0,442.0,885.0
3,ALLEN,3904422,4223.0,3625.0,4223.0
4,ALLEN,3904521,1220.0,1153.0,1220.0


In [54]:
enroll3 = enroll.merge(enroll2, how = "left", on = "COUNTY")

In [55]:
enroll3

,COUNTY,leaid,Enrollment_1,ENROLLMENT,district_enroll,county_enroll
0,ADAMS,3900537,932.0,879.0,932.0,4976.0
1,ADAMS,3906190,4044.0,3612.0,4044.0,4976.0
2,ALLEN,3904388,885.0,442.0,885.0,16319.0
3,ALLEN,3904422,4223.0,3625.0,4223.0,16319.0
4,ALLEN,3904521,1220.0,1153.0,1220.0,16319.0
...,...,...,...,...,...,...
582,WOOD,3905071,1005.0,934.0,1005.0,20781.0
583,WOOD,3905072,1736.0,1541.0,1736.0,20781.0
584,WYANDOT,3904526,845.0,811.0,845.0,3476.0
585,WYANDOT,3905074,988.0,934.0,988.0,3476.0


In [56]:
## merging back to the original data
k12_final = k12_1.merge(enroll3, how = "outer", on = ["COUNTY", "leaid"])

In [57]:
k12_final.isnull().sum()

schnam                               0
leaid                                0
gslo                                 9
gshi                                 9
openEnroll                         112
level                                9
MultiBdy                             0
COUNTY                               0
ENROLLMENT_x                        75
DistrictName                         0
PhysicalCity                         0
Enrollment_1_x                       0
DATE                               164
TeachingMethod                       0
SportsParticipation                  0
OnlineInstructionIncrease            0
NetworkInvestment                    0
HardwareInvestment                   0
StaffMaskPolicy                      0
StudentMaskPolicy                    0
StudentIllnessReturnPolicy           0
StudentIsolationArea                 0
SchoolTemporaryShutdown              0
ParentOptOutClassroomTeachin_55      0
LastVerifiedDate                   164
OpenDateGrouped          

In [ ]:
k12_final.to_csv("cleaned_OH_K12.csv")

# part 2 - data "wrangling"

In [73]:
k12 = pd.read_csv("cleaned_OH_K12.csv", parse_dates = ['DATE', 'LastVerifiedDate', 'OpenDateGrouped'], index_col = 0)

In [74]:
k12.shape

(2782, 30)

In [75]:
k12.groupby(['COUNTY', 'TeachingMethod', 'county_enroll'])['district_enroll'].sum()

COUNTY   TeachingMethod  county_enroll
ADAMS    Hybrid          4976.0           26128.0
ALLEN    Hybrid          16319.0          24821.0
         On Premises     16319.0          47457.0
         Online Only     16319.0           1670.0
ASHLAND  Hybrid          7452.0           22984.0
                                           ...   
WOOD     Hybrid          20781.0          64672.0
         On Premises     20781.0           8922.0
         Online Only     20781.0          15100.0
         Pending         20781.0           3700.0
WYANDOT  Hybrid          3476.0           11881.0
Name: district_enroll, Length: 226, dtype: float64

In [76]:
k12.sort_values('COUNTY').head(5)

,schnam,leaid,gslo,gshi,openEnroll,level,MultiBdy,COUNTY,ENROLLMENT_x,DistrictName,...,StudentIllnessReturnPolicy,StudentIsolationArea,SchoolTemporaryShutdown,ParentOptOutClassroomTeachin_55,LastVerifiedDate,OpenDateGrouped,Enrollment_1_y,ENROLLMENT_y,district_enroll,county_enroll
2648,North Adams High School,3906190,7,12,0.0,3.0,0,ADAMS,541.0,Adams County Ohio Valley School District,...,Yes,Yes,Never closed,Yes,2021-01-15,2020-08-26,4044.0,3612.0,4044.0,4976.0
2653,West Union High School,3906190,7,12,0.0,3.0,0,ADAMS,570.0,Adams County Ohio Valley School District,...,Yes,Yes,Never closed,Yes,2021-01-15,2020-08-26,4044.0,3612.0,4044.0,4976.0
2652,West Union Elementary School,3906190,PK,6,0.0,1.0,0,ADAMS,775.0,Adams County Ohio Valley School District,...,Yes,Yes,Never closed,Yes,2021-01-15,2020-08-26,4044.0,3612.0,4044.0,4976.0
3,Manchester High School,3900537,7,12,0.0,3.0,0,ADAMS,374.0,Manchester Local School District,...,Yes,Yes,Never closed,Yes,2021-01-13,2020-08-26,932.0,879.0,932.0,4976.0
4,Manchester Elementary School,3900537,PK,6,0.0,1.0,0,ADAMS,505.0,Manchester Local School District,...,Yes,Yes,Never closed,Yes,2021-01-13,2020-08-26,932.0,879.0,932.0,4976.0


In [77]:
k12[~k12.duplicated(['COUNTY', 'TeachingMethod', 'leaid', 'district_enroll', 'county_enroll'])]\
.groupby(['COUNTY', "TeachingMethod", 'county_enroll'])['district_enroll'].sum().reset_index()

,COUNTY,TeachingMethod,county_enroll,district_enroll
0,ADAMS,Hybrid,4976.0,4976.0
1,ALLEN,Hybrid,16319.0,7816.0
2,ALLEN,On Premises,16319.0,7668.0
3,ALLEN,Online Only,16319.0,835.0
4,ASHLAND,Hybrid,7452.0,5418.0
...,...,...,...,...
221,WOOD,Hybrid,20781.0,12937.0
222,WOOD,On Premises,20781.0,2974.0
223,WOOD,Online Only,20781.0,3020.0
224,WOOD,Pending,20781.0,1850.0


In [78]:
enroll_teach = k12[~k12.duplicated(['COUNTY', 'TeachingMethod', 'leaid', 'district_enroll', 'county_enroll'])]\
.groupby(['COUNTY', "TeachingMethod", 'county_enroll'])['district_enroll'].sum().reset_index()
enroll_teach.rename(columns = {"district_enroll":"total_teaching"},inplace = True)

In [79]:
enroll_teach['prop_teaching'] = enroll_teach['total_teaching'].div(enroll_teach['county_enroll'])
# alt: .apply(lambda x: x/x.sum()) within groupby


In [80]:
enroll_teach

,COUNTY,TeachingMethod,county_enroll,total_teaching,prop_teaching
0,ADAMS,Hybrid,4976.0,4976.0,1.000000
1,ALLEN,Hybrid,16319.0,7816.0,0.478951
2,ALLEN,On Premises,16319.0,7668.0,0.469882
3,ALLEN,Online Only,16319.0,835.0,0.051167
4,ASHLAND,Hybrid,7452.0,5418.0,0.727053
...,...,...,...,...,...
221,WOOD,Hybrid,20781.0,12937.0,0.622540
222,WOOD,On Premises,20781.0,2974.0,0.143111
223,WOOD,Online Only,20781.0,3020.0,0.145325
224,WOOD,Pending,20781.0,1850.0,0.089024


In [69]:
enroll_teach.pivot(index = ["COUNTY", 'county_enroll'], columns = 'TeachingMethod', values = 'prop_teaching').reset_index()\
.drop(columns = ['Other', 'Pending', 'Unknown'], axis = 1).fillna(0)

TeachingMethod,COUNTY,county_enroll,Hybrid,On Premises,Online Only
0,ADAMS,4976.0,1.000000,0.000000,0.000000
1,ALLEN,16319.0,0.478951,0.469882,0.051167
2,ASHLAND,7452.0,0.727053,0.000000,0.129764
3,ASHTABULA,12031.0,0.633281,0.261574,0.105145
4,ATHENS,7412.0,0.640448,0.000000,0.359552
...,...,...,...,...,...
81,WASHINGTON,5312.0,0.876694,0.123306,0.000000
82,WAYNE,14429.0,0.557835,0.252062,0.000000
83,WILLIAMS,6076.0,0.202765,0.713957,0.000000
84,WOOD,20781.0,0.622540,0.143111,0.145325


In [70]:
wide_teach = enroll_teach.pivot(index = ["COUNTY", 'county_enroll'], columns = 'TeachingMethod', values = 'prop_teaching').reset_index().fillna(0)
wide_teach.rename_axis(None, axis = 1, inplace = True)

In [71]:
wide_teach.loc[:, 'Hybrid':'Online Only'].values.tolist()[0]

[1.0, 0.0, 0.0]

In [72]:
majority_teaching = []

In [73]:
for i in wide_teach.loc[:, 'Hybrid':'Online Only'].values.tolist():
    post = ''
    tmp = max(i)
    if tmp == i[0]:
        post = 'Hybrid'
    elif tmp == i[1]:
        post = 'On_Premises'
    elif tmp == i[2]:
        post = 'Online_Only'
    elif tmp == i[3]:
        post = 'Other'
    elif tmp == i[4]:
        post = 'Pending'
    else:
        post = 'Unknown'
    majority_teaching.append(post)

In [74]:
wide_teach['majority_teaching'] = majority_teaching

In [77]:
wide_teach.majority_teaching

0          Hybrid
1          Hybrid
2          Hybrid
3          Hybrid
4          Hybrid
         ...     
81         Hybrid
82         Hybrid
83    On_Premises
84         Hybrid
85         Hybrid
Name: majority_teaching, Length: 86, dtype: object

In [76]:
wide_teach.majority_teaching.value_counts()

Hybrid         59
On_Premises    16
Online_Only    11
Name: majority_teaching, dtype: int64